## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from datetime import date
from pathlib import Path

from chardet.universaldetector import UniversalDetector
import pandas as pd
import sqlalchemy as sqla

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
dataroot = Path('../data').resolve()
!ls {dataroot}

2020-02-19-parcels.csv
2020-02-19-parcels.zip
2020-02-19-real_property_residential_characteristics.csv
2020-02-19-real_property_sales_and_transfers.csv
re-denver.sqlite3


In [5]:
dbeng = sqla.create_engine('sqlite:///' + str(dataroot / 're-denver.sqlite3'))

## Sales
[Data source page](https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-real-property-sales-and-transfers)

In [6]:
sales_csv = dataroot / '2020-02-19-real_property_sales_and_transfers.csv'

In [7]:
!head -n3 {sales_csv}

"SCHEDNUM","RECEPTION_NUM","INSTRUMENT","SALE_YEAR","SALE_MONTHDAY","RECEPTION_DATE","SALE_PRICE","GRANTOR","GRANTEE","CLASS","MKT_CLUS","D_CLASS","D_CLASS_CN","NBHD_1","NBHD_1_CN","PIN"
"0003100014000","2017095534","WD","2017","0720","20170721","12500000","LNR CPI HIGH POINT LLC","ACM HIGH POINT VI LLC","I","","070","DRY FARM LAND","4","GATEWAY N","163408632"
"0003100015000","2017095534","WD","2017","0720","20170721","12500000","LNR CPI HIGH POINT LLC","ACM HIGH POINT VI LLC","I","","070","DRY FARM LAND","4","GATEWAY N","163408641"


In [8]:
# get row count; NOTE: count includes header row
!wc -l {sales_csv}

332667 /home/jovyan/data/2020-02-19-real_property_sales_and_transfers.csv


In [9]:
# read csv; all data types will be strings initially
sales_df = pd.read_csv(sales_csv, encoding='iso-8859-1', dtype='str')

In [10]:
sales_df.shape

(332666, 16)

In [11]:
sales_df.head(3)

,SCHEDNUM,RECEPTION_NUM,INSTRUMENT,SALE_YEAR,SALE_MONTHDAY,RECEPTION_DATE,SALE_PRICE,GRANTOR,GRANTEE,CLASS,MKT_CLUS,D_CLASS,D_CLASS_CN,NBHD_1,NBHD_1_CN,PIN
0,0003100014000,2017095534,WD,2017,0720,20170721,12500000,LNR CPI HIGH POINT LLC,ACM HIGH POINT VI LLC,I,NaN,070,DRY FARM LAND,4,GATEWAY N,163408632
1,0003100015000,2017095534,WD,2017,0720,20170721,12500000,LNR CPI HIGH POINT LLC,ACM HIGH POINT VI LLC,I,NaN,070,DRY FARM LAND,4,GATEWAY N,163408641
2,0003100016000,2017095534,WD,2017,0720,20170721,12500000,LNR CPI HIGH POINT LLC,ACM HIGH POINT VI LLC,I,NaN,070,DRY FARM LAND,4,GATEWAY N,163408659


In [12]:
# lower case all column names
sales_df.columns = map(str.lower, sales_df.columns)

In [13]:
# numeric type for numeric columns
sales_df.sale_price = pd.to_numeric(sales_df.sale_price)
# sales_df.sale_price = sales_df.sale_price.astype(pd.Int64Dtype()) # convert to nullable int

In [14]:
# create new datetime columns; underscore to indicate calculated columns
sales_df['sale_date_'] = pd.to_datetime(sales_df.sale_year + sales_df.sale_monthday, format='%Y%m%d')
sales_df['reception_date_'] = pd.to_datetime(sales_df.reception_date, format='%Y%m%d', errors='coerce')

In [15]:
sales_df.dtypes

schednum                   object
reception_num              object
instrument                 object
sale_year                  object
sale_monthday              object
reception_date             object
sale_price                float64
grantor                    object
grantee                    object
class                      object
mkt_clus                   object
d_class                    object
d_class_cn                 object
nbhd_1                     object
nbhd_1_cn                  object
pin                        object
sale_date_         datetime64[ns]
reception_date_    datetime64[ns]
dtype: object

In [16]:
# create dtype map for sql table; aim for better match to original types
sales_dtypes = {
    'sale_price': sqla.types.Integer(),
    'sale_date_': sqla.types.Date(),
    'reception_date_': sqla.types.Date(),
}

# write to sql table
sales_df.to_sql('sales', dbeng, index=False, if_exists='replace', dtype=sales_dtypes, chunksize=50000)

In [17]:
# compare sql row count with original row count
dbeng.execute('select count(*) from sales').fetchone()

(332666,)

In [18]:
# check sql data types
dbeng.execute('pragma table_info(sales)').fetchall() # sqlite-specific

[(0, 'schednum', 'TEXT', 0, None, 0),
 (1, 'reception_num', 'TEXT', 0, None, 0),
 (2, 'instrument', 'TEXT', 0, None, 0),
 (3, 'sale_year', 'TEXT', 0, None, 0),
 (4, 'sale_monthday', 'TEXT', 0, None, 0),
 (5, 'reception_date', 'TEXT', 0, None, 0),
 (6, 'sale_price', 'INTEGER', 0, None, 0),
 (7, 'grantor', 'TEXT', 0, None, 0),
 (8, 'grantee', 'TEXT', 0, None, 0),
 (9, 'class', 'TEXT', 0, None, 0),
 (10, 'mkt_clus', 'TEXT', 0, None, 0),
 (11, 'd_class', 'TEXT', 0, None, 0),
 (12, 'd_class_cn', 'TEXT', 0, None, 0),
 (13, 'nbhd_1', 'TEXT', 0, None, 0),
 (14, 'nbhd_1_cn', 'TEXT', 0, None, 0),
 (15, 'pin', 'TEXT', 0, None, 0),
 (16, 'sale_date_', 'DATE', 0, None, 0),
 (17, 'reception_date_', 'DATE', 0, None, 0)]

## Characteristics
[Data source page](https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-real-property-residential-characteristics)

In [19]:
chars_csv = dataroot / '2020-02-19-real_property_residential_characteristics.csv'

In [20]:
!head -n 3 {chars_csv}

"PIN","SCHEDNUM","CD","OFCARD","OWNER","CO_OWNER","OWNER_NUM","OWNER_DIR","OWNER_ST","OWNER_TYPE","OWNER_APT","OWNER_CITY","OWNER_STATE","OWNER_ZIP","SITE_NBR","SITE_DIR","SITE_NAME","SITE_MODE","SITE_MORE","TAX_DIST","PROP_CLASS","PROPERTY_CLASS","ZONE10","D_CLASS_CN","LAND_SQFT","AREA_ABG","BSMT_AREA","FBSMT_SQFT","GRD_AREA","STORY","STYLE_CN","BED_RMS","FULL_B","HLF_B","CCYRBLT","CCAGE_RM","UNITS","ASMT_APPR_LAND","TOTAL_VALUE","ASDLAND","ASSESS_VALUE","ASMT_TAXABLE","ASMT_EXEMPT_AMT","NBHD_1","NBHD_1_CN","LEGL_DESCRIPTION"
"160431419","0004100122000","01","01","REITHER,JOHN FAMILY TRUST  ","","17675 ","E","64TH","AVE","A","DENVER","CO","80249-6409","17675","E","64TH","AVE","","DENVER","1112","Single Family Residential","GTWY        ","SINGLE FAMILY","0","1269","1269","634","0","1","ONE-STORY","3","1","1","1956","0","001","0","345000","0","24668","24670","0","4","GATEWAY N","T3 R66 S4 SW/4 E 660FT EXC S   30FT OF SD SEC IMPS ONLY SEE   00041-00-127 FOR AGRI LAND"
"160431427","000410

In [21]:
# get a line count; count includes header row
!wc -l {chars_csv}

208501 /home/jovyan/data/2020-02-19-real_property_residential_characteristics.csv


In [22]:
# read csv; all data types will be strings initially
chars_df = pd.read_csv(chars_csv, encoding='iso-8859-1', dtype='str')

In [23]:
chars_df.shape

(208500, 46)

In [24]:
chars_df.head(3)

,PIN,SCHEDNUM,CD,OFCARD,OWNER,CO_OWNER,OWNER_NUM,OWNER_DIR,OWNER_ST,OWNER_TYPE,OWNER_APT,OWNER_CITY,OWNER_STATE,OWNER_ZIP,SITE_NBR,SITE_DIR,SITE_NAME,SITE_MODE,SITE_MORE,TAX_DIST,PROP_CLASS,PROPERTY_CLASS,ZONE10,D_CLASS_CN,LAND_SQFT,AREA_ABG,BSMT_AREA,FBSMT_SQFT,GRD_AREA,STORY,STYLE_CN,BED_RMS,FULL_B,HLF_B,CCYRBLT,CCAGE_RM,UNITS,ASMT_APPR_LAND,TOTAL_VALUE,ASDLAND,ASSESS_VALUE,ASMT_TAXABLE,ASMT_EXEMPT_AMT,NBHD_1,NBHD_1_CN,LEGL_DESCRIPTION
0,160431419,0004100122000,01,01,"REITHER,JOHN FAMILY TRUST",NaN,17675,E,64TH,AVE,A,DENVER,CO,80249-6409,17675,E,64TH,AVE,NaN,DENVER,1112,Single Family Residential,GTWY,SINGLE FAMILY,0,1269,1269,634,0,1,ONE-STORY,3,1,1,1956,0,001,0,345000,0,24668,24670,0,4,GATEWAY N,T3 R66 S4 SW/4 E 660FT EXC S 30FT OF SD SEC ...
1,160431427,0004100123000,01,01,"GARRISON,DONALD H",NaN,17675,E,64TH,AVE,NaN,DENVER,CO,80249-6409,17675,E,64TH,AVE,NaN,DENVER,1112,Single Family Residential,GTWY,SINGLE FAMILY,0,1682,0,0,0,1,ONE-STORY,3,1,0,1975,0,001,0,271600,0,19419,19420,0,4,GATEWAY N,T3 R66 S4 SW/4 E 660FT EXC S 30FT OF SD SEC ...
2,163087844,0010201002000,01,01,VACHOVSKY FAMILY TRUST,NaN,Post Office BOX,NaN,8274,NaN,NaN,RANCHO SANTA FE,CA,92067,5845,N,BISCAY,ST,NaN,419C,1114,Rowhouses,C-MU-30,ROWHOUSE,2760,1716,0,0,0,2,END UNIT,3,2,1,2003,0,001,48100,281000,3439,20091,20090,0,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L2


In [25]:
# lower-case column names
chars_df.columns = map(str.lower, chars_df.columns)

In [26]:
# create list of numeric columns
numeric_cols = [
    'land_sqft',
    'area_abg',
    'bsmt_area',
    'fbsmt_sqft',
    'grd_area',
    'story',
    'bed_rms',
    'full_b',
    'hlf_b',
    'ccyrblt',
    'ccage_rm',
    'units',
    'asmt_appr_land',
    'total_value', 
    'asdland', 
    'assess_value',
    'asmt_taxable',
    'asmt_exempt_amt',
    'nbhd_1'
]

# convert numeric columns to numeric data type
for col in numeric_cols:
    chars_df[col] = pd.to_numeric(chars_df[col]) # convert to numeric
    # chars_df[col] = chars_df[col].astype(pd.Int64Dtype()) # convert to nullable int

In [27]:
chars_df.dtypes

pin                  object
schednum             object
cd                   object
ofcard               object
owner                object
co_owner             object
owner_num            object
owner_dir            object
owner_st             object
owner_type           object
owner_apt            object
owner_city           object
owner_state          object
owner_zip            object
site_nbr             object
site_dir             object
site_name            object
site_mode            object
site_more            object
tax_dist             object
prop_class           object
property_class       object
zone10               object
d_class_cn           object
land_sqft           float64
area_abg              int64
bsmt_area             int64
fbsmt_sqft            int64
grd_area              int64
story                 int64
style_cn             object
bed_rms               int64
full_b              float64
hlf_b               float64
ccyrblt             float64
ccage_rm            

In [28]:
# create map of appropriate dtypes for sql table; aim for better match to original types
chars_dtypes = { col: sqla.types.Integer() for col in numeric_cols}

# write to sql
chars_df.to_sql('chars', dbeng, index=False, if_exists='replace', dtype=chars_dtypes, chunksize=50000)

In [29]:
# get row count from sql
dbeng.execute('select count(*) from chars').fetchone()

(208500,)

In [30]:
# get sql 
dbeng.execute('pragma table_info(chars)').fetchall()

[(0, 'pin', 'TEXT', 0, None, 0),
 (1, 'schednum', 'TEXT', 0, None, 0),
 (2, 'cd', 'TEXT', 0, None, 0),
 (3, 'ofcard', 'TEXT', 0, None, 0),
 (4, 'owner', 'TEXT', 0, None, 0),
 (5, 'co_owner', 'TEXT', 0, None, 0),
 (6, 'owner_num', 'TEXT', 0, None, 0),
 (7, 'owner_dir', 'TEXT', 0, None, 0),
 (8, 'owner_st', 'TEXT', 0, None, 0),
 (9, 'owner_type', 'TEXT', 0, None, 0),
 (10, 'owner_apt', 'TEXT', 0, None, 0),
 (11, 'owner_city', 'TEXT', 0, None, 0),
 (12, 'owner_state', 'TEXT', 0, None, 0),
 (13, 'owner_zip', 'TEXT', 0, None, 0),
 (14, 'site_nbr', 'TEXT', 0, None, 0),
 (15, 'site_dir', 'TEXT', 0, None, 0),
 (16, 'site_name', 'TEXT', 0, None, 0),
 (17, 'site_mode', 'TEXT', 0, None, 0),
 (18, 'site_more', 'TEXT', 0, None, 0),
 (19, 'tax_dist', 'TEXT', 0, None, 0),
 (20, 'prop_class', 'TEXT', 0, None, 0),
 (21, 'property_class', 'TEXT', 0, None, 0),
 (22, 'zone10', 'TEXT', 0, None, 0),
 (23, 'd_class_cn', 'TEXT', 0, None, 0),
 (24, 'land_sqft', 'INTEGER', 0, None, 0),
 (25, 'area_abg', 'INTEGE

## Parcels
[source link](https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-parcels)

In [31]:
parcels_csv = dataroot / '2020-02-19-parcels.csv'

In [32]:
!head -n 3 {parcels_csv}

PIN,SCHEDNUM,MAPNUM,BLKNUM,PARCELNUM,APPENDAGE,PARCEL_SOURCE,SYSTEM_START_DATE,OWNER_NAME,OWNER_ADDRESS_LINE1,OWNER_ADDRESS_LINE2,OWNER_ADDR_NBR_PREFIX,OWNER_ADDR_NBR,OWNER_ADDR_NBR_SUFFIX,OWNER_STR_NAME_PRE_MOD,OWNER_STR_NAME_PRE_DIR,OWNER_STR_NAME_PRE_TYPE,OWNER_STR_NAME,OWNER_STR_NAME_POST_TYPE,OWNER_STR_NAME_POST_DIR,OWNER_STR_NAME_POST_MOD,OWNER_UNIT_TYPE,OWNER_UNIT_IDENT,OWNER_CITY,OWNER_STATE,OWNER_ZIP,SITUS_ADDRESS_ID,SITUS_ADDRESS_LINE1,SITUS_ADDRESS_LINE2,SITUS_CITY,SITUS_STATE,SITUS_ZIP,SITUS_ADDR_NBR,SITUS_ADDR_NBR_SUFFIX,SITUS_STR_NAME_PRE_MOD,SITUS_STR_NAME_PRE_DIR,SITUS_STR_NAME_PRE_TYPE,SITUS_STR_NAME,SITUS_STR_NAME_POST_TYPE,SITUS_STR_NAME_POST_DIR,SITUS_STR_NAME_POST_MOD,SITUS_UNIT_TYPE,SITUS_UNIT_IDENT,TAX_DIST,LAND_VALUE,IMPROVEMENT_VALUE,IMPROVEMENTS,PROP_CLASS,TOTAL_VALUE,LAND,D_CLASS,D_CLASS_CN,DCL12,ACT_ZONE,IMP_AREA,CCYRBLT,ASAL_INSTR,SALE_MONTHDAY,SALE_YEAR,SALE_PRICE,RECEPTION_NUM
161001315,0313400005000,03134,00,005,000, ,2010-12-08 00:00:00,PUBLIC SERVICE C

In [33]:
# get a word count
!wc -l {parcels_csv}

232888 /home/jovyan/data/2020-02-19-parcels.csv


In [34]:
# read csv; initally all colums will be strings
parcels_df = pd.read_csv(parcels_csv, encoding='iso-8859-1', dtype='str')

In [35]:
parcels_df.shape

(232887, 61)

In [36]:
parcels_df.head(3)

,PIN,SCHEDNUM,MAPNUM,BLKNUM,PARCELNUM,APPENDAGE,PARCEL_SOURCE,SYSTEM_START_DATE,OWNER_NAME,OWNER_ADDRESS_LINE1,OWNER_ADDRESS_LINE2,OWNER_ADDR_NBR_PREFIX,OWNER_ADDR_NBR,OWNER_ADDR_NBR_SUFFIX,OWNER_STR_NAME_PRE_MOD,OWNER_STR_NAME_PRE_DIR,OWNER_STR_NAME_PRE_TYPE,OWNER_STR_NAME,OWNER_STR_NAME_POST_TYPE,OWNER_STR_NAME_POST_DIR,OWNER_STR_NAME_POST_MOD,OWNER_UNIT_TYPE,OWNER_UNIT_IDENT,OWNER_CITY,OWNER_STATE,OWNER_ZIP,SITUS_ADDRESS_ID,SITUS_ADDRESS_LINE1,SITUS_ADDRESS_LINE2,SITUS_CITY,SITUS_STATE,SITUS_ZIP,SITUS_ADDR_NBR,SITUS_ADDR_NBR_SUFFIX,SITUS_STR_NAME_PRE_MOD,SITUS_STR_NAME_PRE_DIR,SITUS_STR_NAME_PRE_TYPE,SITUS_STR_NAME,SITUS_STR_NAME_POST_TYPE,SITUS_STR_NAME_POST_DIR,SITUS_STR_NAME_POST_MOD,SITUS_UNIT_TYPE,SITUS_UNIT_IDENT,TAX_DIST,LAND_VALUE,IMPROVEMENT_VALUE,IMPROVEMENTS,PROP_CLASS,TOTAL_VALUE,LAND,D_CLASS,D_CLASS_CN,DCL12,ACT_ZONE,IMP_AREA,CCYRBLT,ASAL_INSTR,SALE_MONTHDAY,SALE_YEAR,SALE_PRICE,RECEPTION_NUM
0,161001315,0313400005000,03134,00,005,000,,2010-12-08 00:00:00,PUBLIC SERVICE CO,PO BOX 840,None,None,840,None,None,None,None,PO BOX,None,None,None,None,None,DENVER,CO,80201-0840,None,5401 W CLEAR CREEK DR,None,DENVER,CO,None,5401,None,None,W,None,CLEAR CREEK,DR,None,None,None,None,DENVER,100,0,0,8991,100,3780,099,VCNT LAND,09,S-SU-D,0,None,None,None,None,None,None
1,163314930,0017100019000,00171,00,019,000,,2010-12-08 00:00:00,PARKFIELD II HOMEOWNERS ASSOCIATION,11002 BENTON ST,None,None,11002,None,None,None,None,BENTON,ST,None,None,None,None,WESTMINSTER,CO,80020-3286,231624,5390 N LEWISTON ST,None,DENVER,CO,None,5390,None,None,N,None,LEWISTON,ST,None,None,None,None,DENVER,100,0,0,9199,100,5622,099,VCNT LAND,09,PUD,0,None,WD,1229,2010,None,2010153730
2,162839350,0220433034000,02204,33,034,000,,2010-12-08 00:00:00,CITY & COUNTY OF DENVER,201 W COLFAX AVE DEPT 401,None,None,201,None,None,W,None,COLFAX,AVE,None,None,DEPT,401,DENVER,CO,80211-5330,None,3800 N CLAY ST,None,DENVER,CO,None,3800,None,None,N,None,CLAY,ST,None,None,None,None,DENVER,100,0,0,9139,100,2068,099,VCNT LAND,09,U-SU-C1,0,None,QC,0728,1902,1,0148600175


In [37]:
# lower case coulumn names
parcels_df.columns = map(str.lower, parcels_df.columns)

In [38]:
# construct list of numeric columns
parcels_numeric_cols = [
    'total_value',
    'land',
    'imp_area',
    'sale_price'
]

# convert numeric columns to numeric data type
for col in parcels_numeric_cols:
    # NOTE: 'coerce' will interpret all non-numeric values as Nan
    parcels_df[col] = pd.to_numeric(parcels_df[col], errors='coerce')
    # parcels_df[col] = parcels_df[col].astype(pd.Int64Dtype()) # convert to nullable int

In [39]:
# create new datetime column for sale date; trailing underscore to indicate calculated column
# NOTE: 'coerce' is used here to interpret some garbage values as Nan
parcels_df['sale_date_'] = pd.to_datetime(
    parcels_df.sale_year + parcels_df.sale_monthday, format='%Y%m%d', errors='coerce'
)

In [40]:
parcels_df.dtypes

pin                                 object
schednum                            object
mapnum                              object
blknum                              object
parcelnum                           object
appendage                           object
parcel_source                       object
system_start_date                   object
owner_name                          object
owner_address_line1                 object
owner_address_line2                 object
owner_addr_nbr_prefix               object
owner_addr_nbr                      object
owner_addr_nbr_suffix               object
owner_str_name_pre_mod              object
owner_str_name_pre_dir              object
owner_str_name_pre_type             object
owner_str_name                      object
owner_str_name_post_type            object
owner_str_name_post_dir             object
owner_str_name_post_mod             object
owner_unit_type                     object
owner_unit_ident                    object
owner_city 

In [41]:
# create map for sql data types; aim for better match to original types
parcels_dtypes = { col: sqla.types.Integer() for col in parcels_numeric_cols}
parcels_dtypes.update({'sale_date_': sqla.types.Date()})

# write to sql
parcels_df.to_sql('parcels', dbeng, index=False, if_exists='replace', dtype=parcels_dtypes, chunksize=50000)

In [42]:
# get line count from sql
dbeng.execute('select count(*) from parcels;').fetchone()

(232887,)

In [43]:
# get data types from sql
dbeng.execute('pragma table_info(parcels);').fetchall()

[(0, 'pin', 'TEXT', 0, None, 0),
 (1, 'schednum', 'TEXT', 0, None, 0),
 (2, 'mapnum', 'TEXT', 0, None, 0),
 (3, 'blknum', 'TEXT', 0, None, 0),
 (4, 'parcelnum', 'TEXT', 0, None, 0),
 (5, 'appendage', 'TEXT', 0, None, 0),
 (6, 'parcel_source', 'TEXT', 0, None, 0),
 (7, 'system_start_date', 'TEXT', 0, None, 0),
 (8, 'owner_name', 'TEXT', 0, None, 0),
 (9, 'owner_address_line1', 'TEXT', 0, None, 0),
 (10, 'owner_address_line2', 'TEXT', 0, None, 0),
 (11, 'owner_addr_nbr_prefix', 'TEXT', 0, None, 0),
 (12, 'owner_addr_nbr', 'TEXT', 0, None, 0),
 (13, 'owner_addr_nbr_suffix', 'TEXT', 0, None, 0),
 (14, 'owner_str_name_pre_mod', 'TEXT', 0, None, 0),
 (15, 'owner_str_name_pre_dir', 'TEXT', 0, None, 0),
 (16, 'owner_str_name_pre_type', 'TEXT', 0, None, 0),
 (17, 'owner_str_name', 'TEXT', 0, None, 0),
 (18, 'owner_str_name_post_type', 'TEXT', 0, None, 0),
 (19, 'owner_str_name_post_dir', 'TEXT', 0, None, 0),
 (20, 'owner_str_name_post_mod', 'TEXT', 0, None, 0),
 (21, 'owner_unit_type', 'TEXT', 

# Cleanup

In [44]:
dbeng.execute('vacuum') # compact db